In [1]:
## Pre-Creating knowledge of LOS
import snowflake.connector
import pandas as pd
from pathlib import Path
from dotenv import load_dotenv
import os
load_dotenv(override=True)
from openai import OpenAI





In [2]:

# Snowflake config
snowflake_user = os.getenv('SNOWFLAKE_USER')
snowflake_password = os.getenv('SNOWFLAKE_PASS')
snowflake_account = os.getenv('SNOWFLAKE_ACCOUNTID')
database = os.getenv('SNOWFLAKE_DATABASE')
schema = os.getenv('SNOWFLAKE_SCHEMA')
warehouse = os.getenv('SNOWFLAKE_WAREHOUSE')

def get_3_data():
    """get Introduction LOS Summary"""
    df = {}
    try:
        conn = snowflake.connector.connect(       
            user=snowflake_user,
            password=snowflake_password,
            account=snowflake_account,
            warehouse=warehouse,
            database=database,
            schema=schema,
        )
        
        # run the sql query to get data from snowflake
        cursor = conn.cursor()
        sql = f"SELECT NAMEOFTOPIC, INTRODUCTION, LEARNINGOUTCOME, SUMMARY FROM CFA_COURSES ;"

        results = cursor.execute(sql)
        if results is not None:
            df = pd.DataFrame(results.fetchall())
            df.columns = ['Topic','Introduction', 'Learning outcomes', 'Summary']
        else:
            print('Fail to get data. Try again')
    except NameError as e:
        print(f'Program fail: {e}')
    finally:
        cursor.close()
        conn.close()

    return df

df = get_3_data()


In [27]:

def creat_knowledge(topic, introduction, summary, los):

    client = OpenAI()
    response = client.chat.completions.create(
    model="gpt-3.5-turbo-0125",
    messages=[
        {"role": "system", "content": """You are a helpful assistant to generate techincal note for CFA Refresher Readings articles. Based on the information provided in the prompt by user, create a technical note in Markdown format that summarizes only the key Learning outcomes
          under "Summary of Key Learning Outcome" heading. Be sure to include any tables or equations necessary. Do not provide any introduction or conclusion. The "Summary of Learning outcomes" should have bullets only format which will be used as Knowledge Base for QA prompts."""},
        {"role": "user", "content": f"""## Topic\n{topic}\n\n## Introduction\n{introduction}\n\n## Summary\n{summary}\n\n## Learning Outcomes\n{los}\n"""}
    ]
    )
    # print(response.choices[0].message.content)
    return response

In [24]:

# topic, intro, los, summary = df.loc[0]
# response = creat_knowledge(topic, intro, summary, los)

## Summary

- **Interpreting Interest Rates**: Interest rates can be understood as required rates of return, discount rates, or opportunity costs in financial decision-making processes.

- **Decomposing Interest Rates**: An interest rate comprises a real risk-free rate and premiums compensating investors for various risks such as inflation, default, liquidity, and maturity risk.

- **Effective Annual Rate**: The effective annual rate accounts for compounding within a year and indicates the annual growth of a unit of currency, inclusive of interest on interest.

- **Future and Present Value Calculation**: By using factors like the future value factor ((1 + r)^N) and the present value factor ((1 + r)^-N), one can calculate the future value and present value of a single sum of money, annuities (ordinary and annuity due), perpetuities, and uneven cash flows.

- **Time Line Representation**: A time line can be used to model and solve time value of money problems, visually representing cash 

In [28]:
response_list = []
for i in range(4):
    topic, intro, los, summary = df.loc[i]
    resp = creat_knowledge(topic, intro, summary, los)
    response_list.append([los,resp.choices[0].message.content])



In [26]:
for i in range(4):
    print(response_list[i][1])


## Summary

- **Interest rates:** They represent the required rate of return, discount rate, or opportunity cost. They can be decomposed into a real risk-free rate and various premiums to compensate for different types of risks.
  
- **Effective annual rate (EAR):** It reflects the growth of one unit of currency in a year with compound interest included. It can be calculated based on the stated annual interest rate and the compounding frequency.
  
- **Future value (FV) and Present value (PV):** FV is determined by multiplying PV with the future value factor, while PV is obtained by multiplying FV with the present value factor. These concepts are applicable to single sums, ordinary annuities, annuities due, perpetuities (PV only), and series of uneven cash flows.
  
- **Time value of money problems:** They can be solved for different compounding frequencies using relevant formulas and concepts. Annuities, both ordinary and annuity due, as well as perpetuities, can be calculated using a